In [ ]:
import sys
import datetime as dt

from pathlib import Path

import pandas as pd

from river import metrics
from river import utils
from river.proba import Gaussian

sys.path. insert(1, str(Path().resolve().parent))
from functions.anomaly import GaussianScorer
from functions.proba import MultivariateGaussian
from functions.evaluate import print_stats, progressive_val_predict

In [ ]:
df = pd.read_csv("data/multivariate/Occupancy/room-occupancy-1.test.csv", index_col=0).dropna(axis=0)
df.index = pd.to_timedelta(range(0, len(df)), 'T') + dt.datetime.now().replace(microsecond=0)

df = df.rename(columns={"is_anomaly": "anomaly"})

print(f"{'Real anomalous samples | events | proportion:':<55} "
      f"{sum(df.anomaly):<8} | {sum(df.anomaly.diff().dropna() == 1):<5} | "
      f"{sum(df.anomaly)/len(df.anomaly):.02%}")

In [ ]:
df.plot()

In [ ]:
df.anomaly.plot()

In [ ]:
# CONSTANTS
WINDOW = dt.timedelta(hours=2)
minutes = int(WINDOW.total_seconds()/60)
GRACE_PERIOD=minutes*2/4
LOG_THRESHOLD = -100

f1 = metrics.F1()
recall = metrics.Recall()
prec = metrics.Precision()


model = GaussianScorer(
    utils.TimeRolling(MultivariateGaussian(), period=WINDOW),
    grace_period=GRACE_PERIOD,
    log_threshold=LOG_THRESHOLD
    )


y_pred, change_point =\
    progressive_val_predict(model, df, [f1, recall, prec], print_every=0,
                            protect_anomaly_detector=True,
                            do_predict_log=True,
                            **{"t_a": int(minutes)})

print_stats(df, y_pred, change_point)

In [ ]:
from river import anomaly, preprocessing
from river import metrics
import numpy as np
from river import feature_extraction as fx


class QuantileFilter(anomaly.QuantileFilter):
  def __init__(self, anomaly_detector, q: float, protect_anomaly_detector=True):
        super().__init__(
            anomaly_detector=anomaly_detector,
            protect_anomaly_detector=protect_anomaly_detector,
            q=q
        )
  def predict_one(self, *args):
    score = self.score_one(*args)
    return score >= (self.quantile.get() or np.inf)
    
    
model = (
    preprocessing.StandardScaler() |
    QuantileFilter(
        anomaly.OneClassSVM(),
    q=0.50
    )
    )

f1 = metrics.F1()
recall = metrics.Recall()
prec = metrics.Precision()

y_pred, change_point =\
    progressive_val_predict(model, df, [f1, recall, prec], print_every=10000)
    
print_stats(df, y_pred, change_point)

In [ ]:
from river import anomaly, preprocessing
from river import metrics

scaler = preprocessing.MinMaxScaler()

model = (
    preprocessing.MinMaxScaler() |
    QuantileFilter(
        anomaly.HalfSpaceTrees(),
    q=0.50
    )
    )

f1 = metrics.F1()
recall = metrics.Recall()
prec = metrics.Precision()

y_pred, change_point =\
    progressive_val_predict(model, df, [f1, recall, prec], print_every=10000)

print_stats(df, y_pred, change_point)